In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
!pip install dagshub mlflow --quiet
!pip install imbalanced-learn==0.11.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 6.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 46.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.0/700.0 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [3]:

import dagshub
import mlflow

from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn
import xgboost as xgb
from category_encoders import WOEEncoder
import warnings
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
from imblearn.pipeline import Pipeline as ImbPipeline  # ✅ Corrected import
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import shap
import mlflow.data
from mlflow.data.pandas_dataset import PandasDataset
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

In [4]:
import mlflow
from dagshub import dagshub_logger
import os

# Set tracking URI manually
mlflow.set_tracking_uri("https://dagshub.com/nkikn21/IEEE-CIS-Fraud-Detection.mlflow")

# Use your DagsHub credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "nkikn21"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "90ec7031365aea1b6ca271f4236c194e530973c8"

# Optional: set registry if you're using model registry
mlflow.set_registry_uri("https://dagshub.com/nkikn21/IEEE-CIS-Fraud-Detection.mlflow")



In [5]:
# train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
# train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')

In [6]:
test_identity.columns = test_identity.columns.str.replace('-', '_')

print("Updated Test Columns:", test_identity.columns)

Updated Test Columns: Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object')


In [7]:
# train_joined = train_transaction.merge(train_identity, on="TransactionID", how="left")
test_joined = test_transaction.merge(test_identity, on="TransactionID", how="left")

In [12]:
test_joined['TransactionDay'] = test_joined['TransactionDT'] // (24*60*60)
X_test = test_joined.drop(columns=['isFraud'], errors='ignore')  # Ignore errors in case 'isFraud' isn't in test_joined


**XGBoost**

In [20]:
import mlflow
logged_model = 'runs:/3c97758d9a6c4869b3c466f3aa0db78a/XGB_pipeline'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
preds = loaded_model.predict(X_test)

In [21]:
submission = pd.DataFrame({
    "TransactionID": X_test["TransactionID"],  # or whatever unique ID column you use
    "isFraud": preds
})

# Save submission
submission.to_csv("submission.csv", index=False)

**LGBM**

In [10]:
import mlflow
logged_model = 'runs:/0b7da4f2942141959bb91cb7e79cfa0a/LGBM_pipeline'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
preds = loaded_model.predict(X_test)

In [13]:
submission = pd.DataFrame({
    "TransactionID": X_test["TransactionID"],  # or whatever unique ID column you use
    "isFraud": preds
})

# Save submission
submission.to_csv("submission.csv", index=False)

**Decision Tree**

In [15]:
import mlflow
logged_model = 'runs:/31f2a20112224ef1902a2d509f916550/Decision_Tree_pipeline'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
preds = loaded_model.predict(X_test)

In [16]:
submission = pd.DataFrame({
    "TransactionID": X_test["TransactionID"],  # or whatever unique ID column you use
    "isFraud": preds
})

# Save submission
submission.to_csv("submission.csv", index=False)

**Random Forest**

In [17]:
import mlflow
logged_model = 'runs:/b5cb90ec968d424f8fd20893c0b5991d/Random_Forest_pipeline'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
preds = loaded_model.predict(X_test)

In [18]:
submission = pd.DataFrame({
    "TransactionID": X_test["TransactionID"],  # or whatever unique ID column you use
    "isFraud": preds
})

# Save submission
submission.to_csv("submission.csv", index=False)